In [ ]:
!pip install tensorflow
!pip install gym
!pip install keras
!pip install keras-rl2

In [ ]:
import gym_tetris
import random
from nes_py.wrappers import JoypadSpace
import gym_tetris
from gym_tetris.actions import MOVEMENT

In [ ]:
env = gym_tetris.make('TetrisA-v0')
env = JoypadSpace(env, MOVEMENT)
states = env.observation_space.shape[0]
actions = env.action_space.n

In [ ]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        n_state, reward, done, info = env.step(env.action_space.sample())
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

In [ ]:
import numpy as np
import tensorflow
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_model(states, actions):
    model = tensorflow.keras.models.Sequential()
    model.add(Flatten(input_shape=(1, 1, 240, 256, 3)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
model = build_model(states, actions)

In [ ]:
model.summary()

In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
tensorflow.keras.layers.Flatten(input_shape=(1, 1, 240, 256, 3))

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

In [ ]:
_ = dqn.test(env, nb_episodes=15, visualize=True)